In [1]:
from diffusers.models import AutoencoderKL
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [6]:
import os

In [7]:
# os.environ["MODEL"] = "C:/Users/ASUS/.cache/huggingface/hub/models--cagliostrolab--animagine-xl-3.1"

In [8]:
# HF_TOKEN = ''

In [9]:
import logging

In [10]:
logger = logging.getLogger(__name__)

In [11]:
def load_pipeline(model_name):
    vae = AutoencoderKL.from_pretrained(
        "madebyollin/sdxl-vae-fp16-fix",
        torch_dtype=torch.float16,
    )
    pipeline = (
        StableDiffusionXLPipeline.from_single_file
        if MODEL.endswith(".safetensors")
        else StableDiffusionXLPipeline.from_pretrained
    )
    
    pipe = pipeline(
        model_name,
        vae=vae,
        torch_dtype=torch.float16,
        custom_pipeline="lpw_stable_diffusion_xl",
        use_safetensors=True,
        add_watermarker=False,
        use_auth_token=HF_TOKEN,
    )

    pipe.to('cuda')
    return pipe


In [12]:
MODEL = "cagliostrolab/animagine-xl-3.1"

In [1]:
 if torch.cuda.is_available():
    pipe = load_pipeline(MODEL)
    pipe.to('cuda')
    logger.info("Loaded on Device!")
    

NameError: name 'torch' is not defined

In [14]:
def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

In [19]:
import torch
import json
import utils
import time

In [ ]:
%%time
def simple_generate(prompt, pipe, device='cuda'):
    # if torch.cuda.is_available() and device == 'cuda':
    #     pipe = load_pipeline(model)
    #     logger.info("Loaded on Device!")
    # else:
    #     logger.error("CUDA is not available. The model cannot be tested on CPU.")
    #     return

    try:
         # Enable mixed precision for faster inference
        with torch.cuda.amp.autocast():
            # Generate images
            result = pipe(prompt=prompt, num_inference_steps=20, guidance_scale=7.5)
            images = result.images


        # Save images with unique filenames
        image_paths = []
        timestamp = int(time.time())
        for i, image in enumerate(images):
            path = f"generated_image_{timestamp}_{i}.png"
            image.save(path)
            image_paths.append(path)
            logger.info(f"Generated image saved at: {path}")

        return image_paths

    except Exception as e:
        logger.exception(f"An error occurred: {e}")
        raise
    finally:
        free_memory()

# Example usage
if __name__ == "__main__":
    prompt = "A detailed 3D model of a character inspired by Paimon from Genshin Impact, designed for a transparent background. The character should feature short, silvery-white hair, star-themed motifs, and an elegant celestial outfit in white, blue, and gold. Render the model in a floating pose with arms open and a cheerful expression. Ensure no background elements are present, leaving the background completely transparent (alpha channel)."

    image_paths = simple_generate(prompt, pipe)
    for path in image_paths:
        print(f"Generated image saved at: {path}")

<timed exec>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors


  0%|          | 0/20 [00:00<?, ?it/s]